In [1]:
from __future__ import division
import operator
import nltk
import string

In [2]:
def isPunct(word):
    return len(word) == 1 and word in string.punctuation

def isNumeric(word):
    try:
        float(word) if '.' in word else int(word)
        return True
    except ValueError:
        return False

In [3]:
class RakeKeywordExtractor:

    def __init__(self):
        self.stopwords = set(nltk.corpus.stopwords.words())
        self.top_fraction = 1 # consider top third candidate keywords by score

    def _generate_candidate_keywords(self, sentences):
        phrase_list = []
        for sentence in sentences:
            #Replaces all StopWords by "|" and lowercases all words
            words = map(lambda x: "|" if x in self.stopwords else x,nltk.word_tokenize(sentence.lower()))
            ##words => NONSTOPWORD | NONSTOPWORD NONSTOPWORD NONSTOPWORD | | NONSTOPWORD | NONSTOPWORD NONSTOPWORD |
            phrase = []
            for word in words:
                if word == "|" or isPunct(word):
                    if len(phrase) > 0:
                        phrase_list.append(phrase)#Got At least 1 NonStopWord
                    phrase = []#Prepare for Next Continous NonStopWords
                else:
                    phrase.append(word)#NonStopWord
        return phrase_list

    def _calculate_word_scores(self, phrase_list):
        word_freq = nltk.FreqDist()
        word_degree = nltk.FreqDist()
        for phrase in phrase_list:
            #degree = len(list(filter(lambda x: not isNumeric(x), phrase))) - 1#Number of Words in the Phrase-1
            for word in phrase:
                word_freq[word] += 1
                word_degree[word]+=1
            for word in word_freq.keys():
                word_degree[word] = word_degree[word] + word_freq[word] # itself
        # word score = deg(w) / freq(w)
        word_scores = {}
        for word in word_freq.keys():
            word_scores[word] = word_degree[word] / word_freq[word]
        return word_scores

    def _calculate_phrase_scores(self, phrase_list, word_scores):
        phrase_scores = {}
        for phrase in phrase_list:
            phrase_score = 0
            for word in phrase:
                phrase_score += word_scores[word]
                phrase_scores[" ".join(phrase)] = phrase_score
        return phrase_scores
    
    def extract(self, text, incl_scores=False):
        sentences = nltk.sent_tokenize(text)
        phrase_list = self._generate_candidate_keywords(sentences)
        word_scores = self._calculate_word_scores(phrase_list)
        phrase_scores = self._calculate_phrase_scores(
        phrase_list, word_scores)
        sorted_phrase_scores = sorted(phrase_scores.items(),
        key=operator.itemgetter(1), reverse=True)
        n_phrases = len(sorted_phrase_scores)
        if incl_scores:
            return sorted_phrase_scores[0:int(n_phrases/self.top_fraction)]
        else:
            return map(lambda x: x[0],sorted_phrase_scores[0:int(n_phrases/self.top_fraction)])

In [4]:
def explainCandidateGeneration():
    rake=RakeKeywordExtractor()
    testSent="This is a test sentence to explain the candidate generation strategy for automatic key phrase extraction."
    cands=rake._generate_candidate_keywords([testSent])
    print("Sentence ~~> ")
    print(testSent)
    print()
    print("Generated Cadidates ~~> ")
    print(cands)

In [5]:
explainCandidateGeneration()

Sentence ~~> 
This is a test sentence to explain the candidate generation strategy for automatic key phrase extraction.

Generated Cadidates ~~> 
[['test', 'sentence'], ['explain'], ['candidate', 'generation', 'strategy'], ['automatic', 'key', 'phrase', 'extraction']]


In [6]:
def test():
    rake = RakeKeywordExtractor()
    keywords = rake.extract("""
    Compatibility of systems of linear constraints over the set of natural 
    numbers. Criteria of compatibility of a system of linear Diophantine 
    equations, strict inequations, and nonstrict inequations are considered. 
    Upper bounds for components of a minimal set of solutions and algorithms 
    of construction of minimal generating sets of solutions for all types of 
    systems are given. These criteria and the corresponding algorithms for 
    constructing a minimal supporting set of solutions can be used in solving 
    all the considered types of systems and systems of mixed types.
    """, incl_scores=True)
    print(keywords)
  
if __name__ == "__main__":
    test()

[('linear diophantine equations', 84.0), ('linear constraints', 63.0), ('natural numbers', 62.0), ('strict inequations', 51.5), ('nonstrict inequations', 50.5), ('minimal generating sets', 49.666666666666664), ('upper bounds', 46.0), ('minimal supporting set', 45.33333333333333), ('minimal set', 36.333333333333336), ('compatibility', 32.0), ('system', 28.0), ('corresponding algorithms', 26.0), ('components', 22.0), ('considered types', 21.833333333333332), ('criteria', 21.0), ('set', 20.666666666666668), ('construction', 18.0), ('algorithms', 15.0), ('solutions', 14.666666666666666), ('considered', 14.5), ('systems', 13.75), ('given', 13.0), ('constructing', 10.0), ('mixed types', 9.333333333333332), ('types', 7.333333333333333), ('used', 7.0), ('solving', 6.0)]
